# Textract Anazlyze API Invoke with human in the loop

In [ ]:
import boto3
import uuid
import time
import re
import pprint
import json
pp = pprint.PrettyPrinter(indent=4)

In [ ]:
# Amazon Textract client
textract = boto3.client('textract')

# Amazon S3 client 
s3 = boto3.client('s3')

In [ ]:
humanLoopName = str(uuid.uuid4())

Enter the name of the S3 bucket you craeted 

In [ ]:
bucket="<Your S3 bucket name>"

Enter the flow definition ARN by copying the flow definition arn you craeted here 

In [ ]:
humanLoopConfig = {
    'FlowDefinitionArn':"<enter your flow definition arn>",
    'HumanLoopName':humanLoopName, 
    'DataAttributes': { 'ContentClassifiers': [ 'FreeOfPersonallyIdentifiableInformation' ]}
}

enter the bucket name in Bucket and sample document name in Name

In [ ]:
response = textract.analyze_document(
        Document={'S3Object': {'Bucket': bucket, 'Name':  "samplecheck.PNG"}},
        FeatureTypes=["FORMS"], 
        HumanLoopConfig=humanLoopConfig
    )

In [ ]:
print(response)

Paste the workteam arn below which you copied from craeting private workteam or 
you can find it in this link https://console.aws.amazon.com/sagemaker/groundtruth?region=us-east-1#/labeling-workforces

In [ ]:
WORKTEAM_ARN= "<your private workteam arn>"

In [ ]:
# Amazon SageMaker client
sagemaker = boto3.client('sagemaker')

# Amazon Augment AI (A2I) client
a2i = boto3.client('sagemaker-a2i-runtime')

In [ ]:


workteamName = WORKTEAM_ARN[WORKTEAM_ARN.rfind('/') + 1:]
print("Navigate to the private worker portal and do the tasks. Make sure you've invited yourself to your workteam!")
print('https://' + sagemaker.describe_workteam(WorkteamName=workteamName)['Workteam']['SubDomain'])



In [ ]:
completed_human_loops = []
resp = a2i.describe_human_loop(HumanLoopName=humanLoopName)
print(f'HumanLoop Name: {humanLoopName}')
print(f'HumanLoop Status: {resp["HumanLoopStatus"]}')
#print(f'HumanLoop Output Destination: {resp["HumanLoopOutput"]}')
print('\n')
    
if resp["HumanLoopStatus"] == "Completed":
    completed_human_loops.append(resp)

In [ ]:

for resp in completed_human_loops:
    splitted_string = re.split('s3://' + bucket  + '/', resp['HumanLoopOutput']['OutputS3Uri'])
    output_bucket_key = splitted_string[1]
    print(output_bucket_key)
    response = s3.get_object(Bucket= bucket, Key=output_bucket_key)
    content = response["Body"].read()
    json_output = json.loads(content)
    pp.pprint(json_output)
    print('\n')
